# Developing maximally-symmetric lattice population algorithm

In [ ]:
from polymerist.graphics.plotutils import presize_subplots

max_n_atoms : int = 10_000

records = {}
for mol_name, n_atoms in counts.items():
    n_oligomers = max_n_atoms // n_atoms
    cubic_lattice_dim = np.ceil(n_oligomers**(1/3))

    lattice_info = {
        'n_atoms'  : n_atoms,
        'n_oligomers' : n_oligomers,
        'occupancy' : n_oligomers / cubic_lattice_dim**3,
        'vacancy' : cubic_lattice_dim**3 - n_oligomers,
    }
    records[mol_name] = lattice_info

df = pd.DataFrame.from_dict(records, orient='index')

fig, ax = presize_subplots(nrows=2, ncols=2, scale=10, elongation=1.0)
for colname, axis in zip(df.columns, ax.flatten()):
    heights, bins, patches = axis.hist(df[colname])
    axis.set_xlabel(colname)
    axis.set_ylabel('Counts')

plt.show()

# Playing with lattice generation (i.e. generating non-cubic lattices)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from polymerist.maths.lattices import generate_int_lattice

In [ ]:
from polymerist.graphics import plotutils

coords = np.array([3, 3, 3])
num_points = coords.prod()

lattice = generate_int_lattice(*coords)
plotutils.scatter_3D(lattice)

In [ ]:
A = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [1/2, 1/2, 1/np.sqrt(2)],
])
shear = lattice @ A
plotutils.scatter_3D(shear)

In [ ]:
dims = np.tile(coords, reps=(num_points, 1))
FCC = np.mod(shear, dims-0.99)
plotutils.scatter_3D(FCC)

# Playing with rich progress

In [ ]:
from rich.progress import track, Progress
from time import sleep

with Progress() as progress:

    task1 = progress.add_task("[red]Downloading...", total=1000)
    task2 = progress.add_task("[green]Processing...", total=1000)
    task3 = progress.add_task("[cyan]Cooking...", total=1000)

    while not progress.finished:
        progress.update(task1, advance=0.5)
        progress.update(task2, advance=0.3)
        progress.update(task3, advance=0.9)
        sleep(0.02)


In [ ]:
import time
import random
from rich.progress import (
    BarColumn,
    Progress,
    SpinnerColumn,
    TaskProgressColumn,
    TimeElapsedColumn,
    TimeRemainingColumn,
)

def process(chunks):
    for chunk in chunks:
        time.sleep(0.1)
        yield chunk

chunks = [random.randint(1,20) for _ in range(100)]

progress_columns = (
    SpinnerColumn(),
    "[progress.description]{task.description}",
    BarColumn(),
    TaskProgressColumn(),
    "Elapsed:",
    TimeElapsedColumn(),
    "Remaining:",
    TimeRemainingColumn(),
)

with Progress(*progress_columns) as progress_bar:
    task = progress_bar.add_task("[blue]Downloading...", total=sum(chunks))
    for chunk in process(chunks):
        progress_bar.update(task, advance=chunk)

In [ ]:
import random
import time

from rich.live import Live
from rich.table import Table


def generate_table() -> Table:
    """Make a new table."""
    table = Table()
    table.add_column("ID")
    table.add_column("Value")
    table.add_column("Status")

    for row in range(random.randint(2, 6)):
        value = random.random() * 100
        table.add_row(
            f"{row}", f"{value:3.2f}", "[red]ERROR" if value < 50 else "[green]SUCCESS"
        )
    return table


with Live(generate_table(), refresh_per_second=4) as live:
    for _ in range(40):
        time.sleep(0.4)
        live.update(generate_table())

In [ ]:
from dataclasses import dataclass
from rich.console import Console, ConsoleOptions, RenderResult
from rich.table import Table

@dataclass
class Student:
    id: int
    name: str
    age: int
    def __rich_console__(self, console: Console, options: ConsoleOptions) -> RenderResult:
        yield f"[b]Student:[/b] #{self.id}"
        my_table = Table("Attribute", "Value")
        my_table.add_row("name", self.name)
        my_table.add_row("age", str(self.age))
        yield my_table

# Playing with cubic/octahedral symmetry groups

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule
from polymerist.monomers import specification

smi = '[Br]-[C]12-[C]3(-[I])-[C]4(-[Cl])-[C]-1(-[N])-[C]1(-[I])-[C]-2(-[Cl])-[C]-3(-[N])-[C]-4-1-[Br]'
print(smi)
mol = Chem.MolFromSmiles(smi, sanitize=True)
mol = Chem.AddHs(mol)
EmbedMolecule(mol)
display(mol)

Chem.MolToMolFile(mol, 'cubane.mol')

In [ ]:
from polymerist.graphics.plotutils import scatter_3D
from polymerist.maths.lattices import generate_int_lattice

d : int = 2
dims = [d, d, d]

lattice = generate_int_lattice(*dims)
COM = lattice.mean(axis=0)
order = np.lexsort(lattice.T)
lattice = lattice[order] - COM # sort lexicographically and translate center of grid to origin

print(order, lattice[order])

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

labels = np.lexsort(lattice.T)
ax.scatter(*lattice.T, color='r')
for i, coords in zip(labels, lattice):
    ax.text(*coords, str(i))

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

In [ ]:
from typing import Generator, Iterable
from itertools import permutations, product as cartesian_product


def sign_alternations(n : int) -> Generator[tuple[int, int, int], None, None]:
    '''Generate every possible n-tuple containing either 1 or -1'''
    for signs in cartesian_product(*([1, -1] for _ in range(n))):
        yield signs

def symmetric_group(n : int) -> Generator[tuple[int, int, int], None, None]:
    '''Generates all permutations of n elements as tuples of the integers [0, n-1]'''
    for perm in permutations(range(n), n):
        yield perm

def orthogonal_basis_transforms(n : int=3) -> Generator[np.ndarray, None, None]:
    '''Generate all matrices in n-dimensions which permute or invert the standard basis vectors'''
    for signs in sign_alternations(n):
        matr = np.diag(signs)
        for perm in symmetric_group(n):
            yield matr[list(perm)]


def cycle_decomposition(permutation : Iterable[int]) -> list[tuple[int]]:
    '''Return the disjoint cycles of a permutation'''
    N = len(permutation)
    assert(N == len(set(permutation))) # uniqueness check
    visited : list[bool] = [False] * N

    cycles = []
    for elem in permutation:
        cycle = []
        while not visited[elem]:
            cycle.append(elem)
            visited[elem] = True
            elem = permutation[elem]
        if cycle:
            cycles.append(tuple(cycle))

    return cycles

In [244]:
g = [el for el in orthogonal_basis_transforms(3)]

In [ ]:
g

In [278]:
from scipy.spatial.transform import Rotation, Slerp

trans = g[-1]

I = Rotation.identity()
rot = Rotation.from_matrix(trans)
full_rot = Rotation.concatenate([I, rot])
interpolator = Slerp(np.linspace(0, 1, num=len(full_rot), dtype=int), full_rot)

In [279]:
from pathlib import Path

dir = Path('gif_frames')
dir.mkdir(exist_ok=True)

n_frames = 20
keyframes = interpolator(np.linspace(0, 1, num=n_frames)).as_matrix()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.autoscale(False)


a = np.copy(lattice)
for i, partial_rot in enumerate(keyframes):
    ax.scatter(*(a @ partial_rot).T)
    ax.set(xlim3d=(-1, 1), xlabel='X')
    ax.set(ylim3d=(-1, 1), ylabel='Y')
    ax.set(zlim3d=(-1, 1), zlabel='Z')
    
    fig.savefig(dir / f'frame_{i}.png')
    ax.clear()
plt.close()

In [285]:
trans

array([[ 0,  0, -1],
       [ 0, -1,  0],
       [-1,  0,  0]])

In [286]:
order

array([0, 4, 2, 6, 1, 5, 3, 7])

In [295]:
from collections import defaultdict, Counter
perms = defaultdict(list)

for i, trans in enumerate(g):
    new = lattice.copy()
    init_order = np.lexsort(lattice.T)
    while True:
        new = new @ trans
        perm = np.lexsort(new.T)
        if np.all(perm == init_order):
            break
        perms[i].append(perm)

print(perms)

defaultdict(<class 'list'>, {1: [array([0, 1, 4, 5, 2, 3, 6, 7])], 2: [array([0, 2, 1, 3, 4, 6, 5, 7])], 3: [array([0, 4, 1, 5, 2, 6, 3, 7]), array([0, 2, 4, 6, 1, 3, 5, 7])], 4: [array([0, 2, 4, 6, 1, 3, 5, 7]), array([0, 4, 1, 5, 2, 6, 3, 7])], 5: [array([0, 4, 2, 6, 1, 5, 3, 7])], 6: [array([4, 5, 6, 7, 0, 1, 2, 3])], 7: [array([2, 3, 6, 7, 0, 1, 4, 5]), array([6, 7, 4, 5, 2, 3, 0, 1]), array([4, 5, 0, 1, 6, 7, 2, 3])], 8: [array([4, 6, 5, 7, 0, 2, 1, 3])], 9: [array([2, 6, 3, 7, 0, 4, 1, 5]), array([3, 1, 7, 5, 2, 0, 6, 4]), array([7, 6, 5, 4, 3, 2, 1, 0]), array([5, 1, 4, 0, 7, 3, 6, 2]), array([4, 6, 0, 2, 5, 7, 1, 3])], 10: [array([1, 3, 5, 7, 0, 2, 4, 6]), array([3, 7, 2, 6, 1, 5, 0, 4]), array([7, 6, 5, 4, 3, 2, 1, 0]), array([6, 4, 2, 0, 7, 5, 3, 1]), array([4, 0, 5, 1, 6, 2, 7, 3])], 11: [array([1, 5, 3, 7, 0, 4, 2, 6]), array([5, 4, 7, 6, 1, 0, 3, 2]), array([4, 0, 6, 2, 5, 1, 7, 3])], 12: [array([2, 3, 0, 1, 6, 7, 4, 5])], 13: [array([4, 5, 0, 1, 6, 7, 2, 3]), array([6, 7,

In [349]:
np.random.shuffle(perm)
print(perm)
cycle_decomposition(perm)

[2 6 3 7 4 1 0 5]


[(2, 3, 7, 5, 1, 6, 0), (4,)]